In [1]:
from typing import Any, TypeVar
from app.domain.value_objects.base import ValueObject
# 泛型
T = TypeVar("T", bound=ValueObject)
id_: T
# bound=ValueObject表示id_必须是ValueObject和他的子类
from typing import TypeVar

a = TypeVar('T')
b = TypeVar('T')

print(a is b)  # 输出 False
print(a == b)  # 输出 False
print(type(a), type(b))  # 都是 <class 'typing.TypeVar'>

False
False
<class 'typing.TypeVar'> <class 'typing.TypeVar'>


In [ ]:
from abc import ABC
from dataclasses import dataclass
from typing import Any, TypeVar

from app.domain.exceptions.base import DomainError
from app.domain.value_objects.base import ValueObject

T = TypeVar("T", bound=ValueObject)


@dataclass(eq=False)
class Entity[T: ValueObject](ABC):
    """
    Base class for domain entities, defined by a unique identity (`id`).
    - `id`: Identity that remains constant throughout the entity's lifecycle.
    - Entities are mutable, but are compared solely by their `id`.
    - Subclasses must set `eq=False` to inherit the equality behavior.
    - Add `kw_only=True` in subclasses to enforce named arguments for clarity & safety.
    """

    id_: T

    def __setattr__(self, name: str, value: Any) -> None:
        """
        Prevents modifying the `id` after it's set.
        Other attributes can be changed as usual.
        """
        if name == "id_" and getattr(self, "id_", None) is not None:
            raise DomainError("Changing entity ID is not permitted.")
        super().__setattr__(name, value)

    def __eq__(self, other: Any) -> bool:
        """
        Two entities are considered equal if they have the same `id`,
        regardless of other attribute values.
        """
        return type(self) is type(other) and other.id_ == self.id_

    def __hash__(self) -> int:
        """
        Generate a hash based on entity type and the immutable `id`.
        This allows entities to be used in hash-based collections and
        reduces the risk of hash collisions between different entity types.
        """
        return hash((type(self), self.id_))


# Hash

假设你有如下实体类：

In [ ]:
# ...existing code...
@dataclass(eq=False, kw_only=True)
class User(Entity[ValueObject]):
    name: str
# ...existing code...


如果你创建两个拥有相同 id_ 的实体：

In [ ]:
user1 = User(id_=UserId(1), name="Alice")
user2 = User(id_=UserId(1), name="Bob")
user3 = User(id_=UserId(2), name="Charlie")

你可以把它们放进 set：



In [ ]:
users = {user1, user2, user3}
print(len(users))  # 输出 2，因为 user1 和 user2 的 id_ 相同，被视为同一个实体

你也可以用实体作为字典的键：



In [ ]:
user_dict = {user1: "data1", user3: "data2"}
print(user_dict[user2])  # 输出 "data1"，因为 user2 和 user1 的 id_ 相同

# dataclass->kw_only

kw_only=True 是 dataclass 的参数，用于强制要求实例化时所有字段必须用关键字参数（即 field=value 形式），不能用位置参数。这可以提升代码的可读性和安全性，避免参数顺序错误。

In [2]:
from dataclasses import dataclass

@dataclass(kw_only=True)
class User:
    id: int
    name: str
    age: int

# 正确用法：必须用关键字参数
user = User(id=1, name="Alice", age=30)

# 错误用法：位置参数会报错
user = User(1, "Alice", 30)  # TypeError

TypeError: User.__init__() takes 1 positional argument but 4 were given

# domin port定义了两个domain需要的外部依赖接口
利用typing.Protocol定义需要依赖的外部功能接口
然后domain->userService中直接用这个未实现的接口
再在unit test中用MagicMock去mock这个接口在userService中的方法
就可以实现脱离hash具体方案的单元测试,测试user对象的创建过程

1. 定义 Domain 层的 Port（接口）
Domain 层通过 typing.Protocol 定义外部依赖接口，只声明需要的功能，不关心具体实现。例如：

In [ ]:
from typing import Protocol
from app.domain.value_objects.raw_password.raw_password import RawPassword

class PasswordHasher(Protocol):
    def hash(self, raw_password: RawPassword) -> bytes: ...
    def verify(self, *, raw_password: RawPassword, hashed_password: bytes) -> bool: ...

2. 在领域服务中依赖 Port
领域服务只依赖接口，不依赖具体实现。这样可以灵活注入不同实现，方便测试和扩展。

class UserService:
    def __init__(self, user_id_generator, password_hasher):
        self._user_id_generator = user_id_generator
        self._password_hasher = password_hasher

    def create_user(self, username, raw_password, role, is_active=True):
        user_id = self._user_id_generator()
        password_hash = self._password_hasher.hash(raw_password)
        # ...创建 User 实体...

3. 单元测试中用 MagicMock Mock Port
测试时用 MagicMock 替换真实实现，模拟接口行为，实现测试与具体方案解耦。

In [ ]:
from unittest.mock import MagicMock, create_autospec
import pytest
from app.domain.ports.password_hasher import PasswordHasher

@pytest.fixture
def password_hasher() -> MagicMock:
    return create_autospec(PasswordHasher)

In [ ]:
def test_creates_active_user_with_hashed_password(password_hasher: MagicMock):
    password_hasher.hash.return_value = b"fake_hash"
    service = UserService(user_id_generator=lambda: 1, password_hasher=password_hasher)
    user = service.create_user("alice", "password", role="USER")
    assert user.password_hash == b"fake_hash"